In [1]:
%run ../../utils/commonImports.py
%run ../../utils/tradingImports.py
%matplotlib inline

from base import supres, BaseStrategy

# Load Data

In [2]:
# dropbox_dir = 'D:\\Dropbox\\My work\\krypl-project'
dropbox_dir = '/Users/david.vesely/Dropbox/My work/krypl-project'
def read_data(pair):
    db = os.path.join(dropbox_dir,'sqlite', 'ploniex-chart-data', f'{pair}.db')
    data = load_trading_data(db, 'chart_data', from_date='2015-01-01', period='30min')
    data['ohlc4'] = (data['open'] + data['close'] + data['high'] + data['low']) / 4
    return data

In [3]:
supports_dict = load_model('supports.pkl')
features = ['support']

# Parallelize

In [4]:
from multiprocessing import Pool
import time
from tqdm import *

def run_parallel(f, args, n_process=10):
    results = []
    with Pool(processes=n_process) as p:
        all_args = list(args)
        with tqdm_notebook(enumerate(p.imap_unordered(f, all_args)), leave=False, total=len(all_args)) as pbar:
            for i, x in pbar:
                results.append(x)
                pbar.update()
                
    return results

In [5]:
def dict_from_list(l):
    d = {}
    for row in l:
        d_inner = d
        for k in row[:-2]:
            if k not in d_inner.keys():
                d_inner[k] = {}
            d_inner = d_inner[k]
        d_inner[row[-2]] = row[-1]
    return d

# Experiment setup

In [6]:
train_pairs = pd.read_csv('pairs_train.tsv', header=None)[0].tolist()
window_sizes = [4, 6, 8, 12, 24, 30, 48]
targets = list(np.linspace(0, 0.1, 11))[1:]
stop_losses = list(np.linspace(0, 0.1, 11))[1:]
returns = list(-np.linspace(0, 0.1, 11))[1:]

loops = len(train_pairs) * len(window_sizes) * len(targets) * len(stop_losses) * len(returns)
loops

210000

# Data With Supports

In [7]:
data_dict = {pair: read_data(pair) for pair in train_pairs}

In [ ]:
from itertools import product

def find_support(i, supports):
    selected = supports[supports['index'] <= i]['support']
    return None if len(selected) == 0 else selected.iloc[-1]


def supports_f(args):
    pair, window_size = args
    supports = supports_dict[pair][window_size]
    data = data_dict[pair]
    s = [find_support(i, supports) for i in range(len(data))]
    return pair, window_size, s

args = product(train_pairs, window_sizes)
results = run_parallel(supports_f, args)
supports_for_data_dict = dict_from_list(results)
save_model(supports_for_data_dict, 'supports-for-data.pkl')

# Try params

In [8]:
supports_for_data_dict = load_model('supports-for-data.pkl')

In [12]:
from itertools import product

def trading_stats(data, pair, target, willing_loss, _return):
    contract_pair = ContractPair.new(*[x.lower() for x in pair.split('_')])
    dm = OhlcDataManager(data[['open', 'low', 'high', 'close']], data[features])
    wallet = {'btc': 100}
    exchange = BackTestExchange(dm, wallet, 0.0025)
    strategy = BaseStrategy(exchange, 
                            dm, 
                            contract_pair, 
                            willing_loss=willing_loss, 
                            target_profit=target, 
                            target_return=_return)
    strategy.trade()
    strategy_stats = strategy.stats(contract_pair['priceContract']).report()
    return strategy_stats


def trading_stats_f(args):
    pair, window_size, target, stop_loss, _return = args
    
    data = data_dict[pair]
    data['support'] = supports_for_data_dict[pair][window_size]
    data = reset_index_hard(data[data['date'] >= '2018-01-01'])
    stats = trading_stats(data, pair, target, stop_loss, _return)
    return pair, window_size, target, stop_loss, _return, stats

In [13]:
args = product(train_pairs, window_sizes, targets, stop_losses, returns)
results = run_parallel(trading_stats_f, args)


param_cols = ['pair', 'window_size', 'target', 'stop_loss', '_return']
stats_cols = ['startAmount', 'numberOfTrades', 'totalProfit', 'avgProfit',
    'winPercentage', 'avgWinTrade', 'avgLossTrade', 'profitFactor',
    'maxDrawdown', 'avgTimeToClose', 'minTimeToClose', 'maxTimeToClose']

result_df = pd.DataFrame(results, columns=param_cols+['stats'])

for c in stats_cols:
    result_df[c] = result_df['stats'].apply(lambda x: x.iloc[:, 0][c])
    
result_df = result_df.drop('stats', axis=1)

rpath = os.path.join(dropbox_dir,'result', 'base-strategy-learn-03.tsv')
write_tsv(result_df, rpath)